In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [18]:
data_transforms = transforms.Compose([
                                 transforms.ToTensor()
])

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms)
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
torch.cuda.get_device_properties(torch.device)

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Ti', major=8, minor=6, total_memory=8191MB, multi_processor_count=38)

In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [23]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [24]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1217 Acc: 0.4691
val Loss: 1.0362 Acc: 0.5197
test Loss: 1.0918 Acc: 0.4895

Epoch 1/24
----------
train Loss: 0.8593 Acc: 0.6192
val Loss: 0.7230 Acc: 0.7060
test Loss: 0.7827 Acc: 0.6526

Epoch 2/24
----------
train Loss: 0.6879 Acc: 0.7278
val Loss: 0.9365 Acc: 0.6247
test Loss: 0.8329 Acc: 0.6526

Epoch 3/24
----------
train Loss: 0.5869 Acc: 0.7647
val Loss: 0.6208 Acc: 0.7822
test Loss: 0.5200 Acc: 0.7842

Epoch 4/24
----------
train Loss: 0.5244 Acc: 0.7941
val Loss: 0.7601 Acc: 0.7717
test Loss: 0.6071 Acc: 0.7947

Epoch 5/24
----------
train Loss: 0.4140 Acc: 0.8341
val Loss: 0.6040 Acc: 0.7874
test Loss: 0.5880 Acc: 0.7684

Epoch 6/24
----------
train Loss: 0.3202 Acc: 0.8831
val Loss: 0.6524 Acc: 0.7743
test Loss: 0.5242 Acc: 0.8316

Epoch 7/24
----------
train Loss: 0.2333 Acc: 0.9208
val Loss: 0.5131 Acc: 0.8241
test Loss: 0.4075 Acc: 0.8368

Epoch 8/24
----------
train Loss: 0.2253 Acc: 0.9095
val Loss: 0.4536 Acc: 0.8373
test Loss: 0.3